## MLE Notebook 3 Theoretical guarantees
### 0. Contents
More MLE examples: linear regression, general linearised regression (logistic and Poisson). 

Using <code>optim()</code>. 

Properties of ML estimators: asymptotically unbiased, asymptotically efficient, consistent, asymptotically normal, invariance. 

### 1. More MLE examples
#### 1.1 Linear regression
I hope we are all familiar with the simplest form of a linear regression. It comprises a vector of response $\underline{y}$ and a vector of independent (or explanatory) variable $\underline{x}$: 
$$y_i=a+bx_i+\epsilon_i$$
where $a$ is the intercept and $b$ is the slope. $i=1, 2, ..., n$. $\epsilon_i$ is the error term, and in simple linear regression, $\epsilon_i$ are assumed to be i.i.d. $N(0,\sigma^2)$. 

Now we have identified the triplets required for parameter estimation: 1) our data $\underline{x}$ and $\underline{y}$, 2) the three parameters of interest: $\{a, b, \sigma^2\}$, 3) and finally the model, which is i.i.d. normal errors. If we rearrange the regression equation such that the error is the subject:
$$\epsilon_i=y_i-a-bx_i$$
Then the likelihood becomes
$$L(\underline{\theta})=f(\epsilon_1\epsilon_2...\epsilon_n)=\prod_{i=1}^nf(\epsilon_i)$$
The first $f$ is the joint density of all the error terms, and the second $f$ is the univariate pdf of an individual error. Now the problem has reverted back to the one we saw in #4.6 yesterday, that we need to find a set of $(\hat{a}, \hat{b}, \hat{\sigma^2})$ that maximises $L(\underline{\theta})$. I probably would delegate this task to R, but in principle you can find the three partial derivatives and solve the system of equations. See today's practical on <code>recapture.csv</code>, also available in a separate notebook. In R, we can use the built-in <code>dnorm(y-a-b*x, mean=0, sd=sigma)</code> instead of coding the normal pdf explicitly. 

In linear regression with normal errors, maximising log-likelihood is equivalent to minimising sum of squares (least squares). 

#### 1.2 Logistic regression
Logistic regression belongs to the family of generalised linear models (GLM). It aims to relate a binary response to a set of explanatory variable(s). For example, in public health a typical binary response will be the state of the patients (dead/alive), or whether a parasite is absent or present in one's body. When I think of a binary variable, I immediately recall the Bernoulli r.v., whose outcome is either 0 or 1. In fact, logistic regression assumes that each binary outcome follows a Bernoulli distribution with its own $p_i$:
$$Y_i\sim Bernoulli(p_i)$$
$i=1, 2, ..., n$. We also understand that $p_i$ is influenced by our explanatory variable $x_i$:
$$p_i=\eta^{-1}(a+bx_i)$$
$a$ and $b$ are the coefficients, similar to the intercept and slope in linear regression. $(a+bx_i)$ is called the linear predictor. The remaining question is, what is $\eta^{-1}$? 
$$\eta^{-1}(a+bx_i)=\frac{e^{a+bx_i}}{1+e^{a+bx_i}}$$
We know from the Bernoulli r.v. that $p_i$ has to be bounded between 0 and 1, but $(a+bx_i)$ can be any real number. That is why we call upon $\eta^{-1}$ to map a real number to $[0, 1]$. $\eta^{-1}$ is called the *expit* transformation, and its inverse $\eta$ is *logit*, hence the name of the model. 

With the triplets identified we can proceed to find the ML estimates for $a$ and $b$. See <code>flowering.txt</code>, also available in a separate notebook. 

#### 1.3 Poisson regression
Poisson regression is another GLM which is often used when the response are counts (e.g. number of mutations/crossovers, malaria cases). As its name suggests, it assumes each response follows a Poisson distribution with its own rate parameter $\lambda_i$:
$$Y_i\sim Poisson(\lambda_i)$$
$i=1, 2, ..., n$. Similar to the logistic case, we need a function to link up $\lambda_i$ with the linear predictor. The constraint here is that $\lambda_i$ must be non-negative due to it being a rate. An appropriate function will be exponential: 
$$\lambda_i=e^{a+bx_i}$$
Note that the inverse of exponential is log. 

In GLM, a link function provides the relationship between the lienar predictor and the mean of the distribution function. Logit and log are link functions in logistic and Poisson GLMs, respectively. I hope these examples help demystify GLMs and their model fitting via <code>glm(y~x, family=)</code>. 

### 2. <code>optim()</code>
<code>optim()</code> is a generic optimisation routines for multivariate functions, similar to what <code>optimize()</code> does in univariate case. Here multivariate refers to the number of parameters rather than observations. It is a so powerful that its help file <code>?optim</code> provides more questions than answers. 
#### 2.1 Inputs
It takes quite a few arguments usually in the following order:
1) The mandatory <code>par=</code> vector specifies the initial condition for the search. If it is a $k$-dimensional function then a vector of length $k$ should be supplied. Avoid starting near the boundary of the parameter space. 
2) Put the function name you wish to optimise under <code>fn=</code>. Mandatory.
3) <code>method=</code> instructs the optimisation algorithm to be used. The default is Nelder-Mead. The choice of algorithm can affect performance and numerical results. Another popular option is <code>method='L-BFGS-B'</code> which takes a box-like constraint (see below).
4) The default algorithm imposes no constraints on the parameter space which is a double-edged sword. When <code>method='L-BFGS-B'</code> is chosen then you must supply <code>lower=</code> and <code>upper=</code> bounds as two vectors (with lengths equal the number of parameters). This algorithm is useful when some parameters cannot go beyond a certain value (e.g. Poisson rates $\lambda_i$ must be non-negative). 
5) Other control options, such as tolerance, can be supplied to the control list <code>control=list((fnscale=-1))</code>. In particular, <code>fnscale=-1</code> means to maximise. The default is minimise. 
6) <code>hessian=T</code> to return the Hessian matrix. Optional but useful. 

#### 2.2 Outputs
It returns a big list: 
1) <code>\$par</code> is the parameter values that maximise the function. 
2) <code>\$value</code> returns the maximised function value.
3) The rest are for performance and convergence checking. If you have asked for the Hessian matrix you may find it in the output as well. 

There are other alternatives to <code>optim()</code>, such as <code>nlm()</code> and <code>nlminb</code>. External packages are available as well. 

### 3. Properties of ML estimators
Below are the theoretical guarantees of ML estimators. They also justify why we are spending a whole week on likelihood. 

#### 3.1 ML estimators are asymptotically unbiased
If we repeat the experient (and ML estimation) for infinitely many times and obtain many $\hat{\theta}$, then the hypothetical average of these $\hat{\theta}$ is $\theta$. When we talk of a hypothetical average we mean expectation: 
$$E[\hat{\theta}]\rightarrow \theta$$
when $n\rightarrow \infty$. 

#### 3.2 ML estimators are asymptotically efficient
Efficiency means the ML estimator $\hat{\theta}$ usually has a lower variance compared to the other estimators (what are the other estimators?). Since it has better use of the data, it produces narrower confidence intervals (C.I.). In particular, $Var[\hat{\theta}]$ reaches the theoretical lower bound when $n\rightarrow \infty$. 

#### 3.3 ML estimators are consistent
The ML estimator $\hat{\theta}$ converges *in probability* to the true parameter $\theta$ when $n\rightarrow \infty$. 

#### 3.4 ML estimators are asymptotically normal
The ML estimator $\hat{\theta}$ is asymptotically distributed as normal with mean equals the true paramter $\theta$. This is closely related to the central limit theorem, and this explains why the magic number 1.96 works in constructing C.I.. Have you ever wondered why a z-test is used in a <code>glm()</code>?

#### 3.5 The invariant principle of ML stimators
If $\hat{\theta}$ is the ML estimator for the parameter $\theta$, then $g(\hat{\theta})$ is the ML estimator for $g(\theta)$. That is, we do not need to recalculate the ML estimator if the transformed parameter is of concern. 